# UTS Authorization and Calling - Python

This notebook is an example of how to start using the UTS REST API from Python 3+

## UTS Account
If you do not have a UTS account:

* You will need to wait for an account to be activated before following this guide.
* Create a bookmark of this page, or [click here](#) to schedule a reminder.
* Follow this link to Sign-up for a [UTS License](https://uts.nlm.nih.gov/license.html).

## UTS API Key
You will need your UTS API key.  To obtain your key:

* In a new window, open the [UTS login](https://uts.nlm.nih.gov//uts.html) page.
* Once you have logged in, click on `My Profile` and copy your API key into the cell below.


In [194]:
UTS_API_KEY = 'XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX'

You'll some basic Python packages to follow the API:

In [185]:
import requests
from lxml import html as lhtml
from urllib.parse import urlencode, urlsplit, urlunsplit

Each UTS session includes three steps:

1. Using your API key to obtain a Ticket Granting Ticket (TGT), which is good for 8 hours.
1. Use the TGT to obtain a single use Server Ticket (ST), which is good for 1 call within the next 5 minutes.
1. Use the ST to make a single API call.

In this session, we will develop a Python class to manage this, so that you merely need to make API calls.

## Step 1 - obtain a Ticket Granting Ticket (TGT)

Let's do the first step manually, and obtain a Ticket Granting Ticket (TGT), first.

Below, we will:
* Do a POST to https://utslogin.nlm.nih.gov/cas/v1/api-key, passing our API Key as data to the call.
* Interpret the results as HTML.
* Parse the TGT from the HTML.

First, let's get the results and show the status code and content type:

In [186]:
session = requests.session()
response = session.post('https://utslogin.nlm.nih.gov/cas/v1/api-key', data={'apikey': UTS_API_KEY})
response.status_code, response.headers['Content-Type']

(201, 'text/html;charset=UTF-8')

Now, we'll parse that as HTML and extract the TGT From the URL

In [187]:
doc = lhtml.fromstring(response.text)
action_uri = doc.xpath("//form/@action")[0]
action_uri

'https://utslogin.nlm.nih.gov/cas/v1/api-key/TGT-9403-OvYn3eeCUwTYEG9SIuqdQIWvShL0qzdIbLSF9KPspqsdJYqEK7-cas'

Now that we have the TGT embedded in the Action URI above, let's move on to step 2:

## Step 2 - obtain a Service Ticket (ST)

We will do this by posting to the action_uri with the service URL http://umlsks.nlm.nih.gov.

In [188]:
r = session.post(action_uri, data={'service': 'http://umlsks.nlm.nih.gov'})
ticket = r.text
ticket

'ST-104315-cTuEsT6qYwZUAKPAblFn-cas'

## Step 3 - make an API call with the Service Ticket (ST)

In this example, we will lookup the CUI by name for diabetic foot.

* We will submit a get request to https://uts-ws.nlm.nih.gov/rest/search/current
* The `ticket` parameter will be the service ticket (ST) obtained above
* The `search` parameter will be the text we want to search for
* The `searchType` will be equal to "exact"

In [189]:
version = 'current'
query_string = 'diabetic foot'
base_search_uri = 'https://uts-ws.nlm.nih.gov/rest/search/%s' % version
search_uri = '%s/?ticket=%s&string=%s&searchType=exact' % (base_search_uri, ticket, query_string)
r = requests.get(search_uri)
search_uri, r, r.headers['Content-Type']


('https://uts-ws.nlm.nih.gov/rest/search/current/?ticket=ST-104315-cTuEsT6qYwZUAKPAblFn-cas&string=diabetic foot&searchType=exact',
 <Response [200]>,
 'application/json;charset=UTF-8')

In [190]:
r.json()

{'pageNumber': 1,
 'pageSize': 25,
 'result': {'classType': 'searchResults',
  'results': [{'name': 'Diabetic Foot',
    'rootSource': 'MTH',
    'ui': 'C0206172',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172'}]}}

## Making this better

Of course, we cannot proceed with this API pattern without some abstraction - let us build an authentication class which will obtain the ST as needed.  Our goal is to be able to make requests without impediment, where for GET requests, a ticket parameter will be added to the URL, and for POST requests, the ticket will be added to the body.  So, we wish this to look no different than requests session object, having methods such as `post()` and `get()`.

This is a big step - we are using pythonic decorators like `@property` to do lazy initialization, and writing a wrapper class without any tests to make sure it works.   Don't worry if this is a little bit complex - the main point here is that you should wrap the UTS REST API so that adding the TGT and ST are pretty much invisible, and you should:

In [191]:
from pprint import pprint

class AuthenticatedSession(object):
    
    def __init__(self, apikey, tgt_uri=None, service_uri=None):
        self.apikey = apikey;
        self.tgt_uri = tgt_uri if tgt_uri is not None else 'https://utslogin.nlm.nih.gov/cas/v1/api-key'
        self.service_uri = service_uri if service_uri is not None else 'http://umlsks.nlm.nih.gov'
        
        # Lazy initialization:
        self.__session = None
        self.__action_uri = None
        
    @property
    def session(self):
        """
        A requests session that we can access as self.session
        """
        if self.__session is None:
            self.__session = requests.session()
        return self.__session
        
    @property
    def action_uri(self):
        """
        A way to memoize (remember) the tgt and uri as a property, accessible like self.action_uri
        """
        if self.__action_uri is None:
            self.__action_uri = self.get_action_uri()
        return self.__action_uri
    
    def get_action_uri(self):
        """
        This will get a TGT embedded in the URI
        """
        response = self.session.post(self.tgt_uri, data={'apikey': self.apikey})
        assert response.status_code == 201
        doc = lhtml.fromstring(response.text)
        actions = doc.xpath('//form/@action')
        assert len(actions) > 0
        return actions[0]
        
    def get_ticket(self):
        """
        This can be used even without the wrapping get and post
        """
        response = self.session.post(self.action_uri, data={'service': self.service_uri})
        assert r.status_code == 200
        return response.text
    
    def get(self, url, params=None, **kwargs):
        if params is not None:
            params['ticket'] = self.get_ticket()
            kwargs['params'] = params
        else:
            uo = urlsplit(url)
            if uo.query:
                newquery += '%s&ticket=%s' % (uo.query, self.get_ticket())
            else:
                newquery = 'ticket=%s' % self.get_ticket()
            url = urlunsplit((uo.scheme, uo.netloc, uo.path, newquery, uo.fragment))
            
        return self.session.get(url, **kwargs)
    
    def post(self, url, data=None, json=None, **kwargs):
        if data is not None:
            data['ticket'] = self.get_ticket()
        elif json is not None:
            json['ticket'] = self.get_ticket()
        return self.__session.post(url, data, json, **kwargs)

## Putting it all together
Now we should be able to access the UTS APIs more simply using AuthenticatedSession() as a wrapper:

In [192]:
session = AuthenticatedSession(apikey=UTS_API_KEY)
session.get_ticket()


'ST-104497-69Md0NW7LgKOBxZiaPS7-cas'

Great!  This gets us tickets without us having to worry about the intervening ticket granting ticket (TGT) and the URI that includes it.   Furthermore, it should be able to wrap calls to the API.  Our previous search results included the content URI for a CUI connected with "diabetic foot", let's use our session wrapper to get that:

In [193]:
response = session.get('https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172')
response.status_code == 200 and response.json()

{'pageCount': 1,
 'pageNumber': 1,
 'pageSize': 25,
 'result': {'atomCount': 76,
  'atoms': 'https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172/atoms',
  'attributeCount': 0,
  'classType': 'Concept',
  'cvMemberCount': 0,
  'dateAdded': '04-12-1994',
  'defaultPreferredAtom': 'https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172/atoms/preferred',
  'definitions': 'https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172/definitions',
  'majorRevisionDate': '03-08-2013',
  'name': 'Diabetic Foot',
  'relationCount': 0,
  'relations': 'NONE',
  'semanticTypes': [{'name': 'Disease or Syndrome',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2017AB/TUI/T047'}],
  'status': 'R',
  'suppressible': False,
  'ui': 'C0206172'}}

## Conclusion

There's a lot more we can do with this, and dress it up to be a real SDK.  We could add support for long-living code, and regenerate the TGT, that is, the action URI, if there was an error.  We can add some code to pull error messages from HTML content.   We'll leave that to you, but assure you that the authorization 